# 处理速度滑冰数据

In [111]:
import pandas as pd
import os

In [112]:
path = "/Volumes/ESD-USB/哈尔滨滑冰测试/速滑辅助鞋垫运动学数据.xlsx"

In [115]:
excelfile = pd.ExcelFile(path)
excelfile.sheet_names

['直道滑行-不穿鞋垫', '直道滑行-穿鞋垫 ', '弯道滑行-不穿鞋垫', '弯道滑行-穿鞋垫', '特征时刻', '特征时刻-20231031']

In [184]:
record_df = pd.read_excel(path, sheet_name="特征时刻")

In [121]:
record_df["文件名"] = record_df["日期"].astype(str) + "_" + \
            record_df["测试序号"].astype(str) + "_" + \
            record_df["姓名"].astype(str) + "_00" + \
            record_df["xsens号"].astype(str) + ".xlsx"

In [ ]:
excel_dir = "/Volumes/ESD-USB/哈尔滨滑冰测试/excel"
for index, row in df.iterows():
    date = row["日期"]
    test_number = row["测试序号"]
    name = row["姓名"]
    xsens_number = row["xsens号"]

    # 拼接文件名
    file_name = f"{date}_{test_number}_{name}-{xsens_number}.xlsx"

    excel_path = os.path.join(excel_dir, file_name)
    
    angle_df = pd.read_excel(excel_path, sheet_name="Joint Angles ZXY")
    angular_vel_df = pd.read_excel(excel_path, sheet_name = "Segment Angular Velocity")
    

In [22]:
excel_path = "/Volumes/ESD-USB/哈尔滨滑冰测试/excel/20231019_1_lininghao-001.xlsx"

angle_df = pd.read_excel(excel_path, sheet_name="Joint Angles ZXY")
angular_vel_df = pd.read_excel(excel_path, sheet_name = "Segment Angular Velocity")



In [41]:
seg_types = ["Left", "Right"]
axis = ["x", "y", "z"]
segments = [" Hip ", " Knee ", " Ankle "]
seg_orientations = ["Flexion/Extension","Abduction/Adduction", "Internal/External Rotation"]
seg_orientations_ankle = ["Dorsiflexion/Plantarflexion","Abduction/Adduction", "Internal/External Rotation"]

for seg_type in seg_types:
    angle_seg_types_colmuns = [seg_type + segment + seg_orientation
                 for seg_type in seg_types 
                 for segment in segments
                 for seg_orientation in ((seg_orientations_ankle if segment.strip()=="Ankle" else seg_orientations))]

In [189]:
# 构建列表头名称
def CreateAngleColumns(seg_type):
    angle_seg_types_colmuns = [seg_type + segment + seg_orientation
                 for segment in segments
                 for seg_orientation in ((seg_orientations_ankle if segment.strip()=="Ankle" else seg_orientations))]
    return angle_seg_types_colmuns

In [80]:
Left_angle_columns = CreateAngleColumns("Left")
Right_angle_columns = CreateAngleColumns("Right")

In [126]:
Left_contact_event = record_df.loc[1, "1直道触冰-左"]
Left_off_event = record_df.loc[1, "1直道离冰-左"]

# 左侧触冰、离冰时髋、膝、踝角度
Left_angle_contact = angle_df[Left_angle_columns].loc[Left_contact_event]
Left_angle_off = angle_df[Left_angle_columns].loc[Left_off_event]

frame = [Left_angle_contact, Left_angle_off]
Left_angles = pd.concat(frame, axis=0)


Right_contact_event = record_df.loc[1, "1直道触冰-右"]
Right_off_event = record_df.loc[1, "1直道离冰-右"]
# 右侧触冰、离冰时髋、膝、踝角度
Right_angle_contact = angle_df[Right_angle_columns].loc[Right_contact_event]
Right_angle_off = angle_df[Right_angle_columns].loc[Right_off_event]

frame = [Right_angle_contact, Right_angle_off]
Right_angles = pd.concat(frame, axis=0)

frame = [Left_angles, Right_angles]
result_frame_1 = pd.concat(frame, axis=0)


In [162]:
# 确定直道一次左脚触冰、离冰和右脚触冰离冰时角度
def GetEachTestResult(record_df, angle_df, record_row, road_number, road_type):
    
    Left_contact_event = record_df.loc[record_row, f"{road_number}{road_type}触冰-左"]
    Left_off_event = record_df.loc[record_row, f"{road_number}{road_type}离冰-左"]
    
    Right_contact_event = record_df.loc[record_row, f"{road_number}{road_type}触冰-右"]
    Right_off_event = record_df.loc[record_row, f"{road_number}{road_type}离冰-右"]


    def GetEventAngles(angle_df, angle_columns, contact_event, off_event):
        # 左侧触冰、离冰时髋、膝、踝角度
        Left_angle_contact = angle_df[angle_columns].loc[contact_event]
        Left_angle_off = angle_df[angle_columns].loc[off_event]

        frame = [Left_angle_contact, Left_angle_off]
        angles = pd.concat(frame, axis=0)
        return angles

    if pd.isna(Left_contact_event):
        print("数据为空")
    else:
        Left_event_angles = GetEventAngles(angle_df, Left_angle_columns, Left_contact_event, Left_off_event)
        Right_event_angles = GetEventAngles(angle_df, Right_angle_columns, Right_contact_event, Right_off_event)

        frame = [Left_event_angles, Right_event_angles]
        Result_each_test = pd.concat(frame, axis=0)

    return Result_each_test

In [186]:
record_row = 1
road_type = "弯道"

# angle_df 
target_data_df = angle_df

Result_test_1 = GetEachTestResult(record_df, target_data_df, record_row, 1, road_type)
Result_test_2 = GetEachTestResult(record_df, target_data_df, record_row, 2, road_type)

frame = [Result_test_1, Result_test_2]
result_one_file_1 = pd.concat(frame, axis=1)

record_row=2
Result_test_1 = GetEachTestResult(record_df, target_data_df, record_row, 1, road_type)
Result_test_2 = GetEachTestResult(record_df, target_data_df, record_row, 2, road_type)

frame = [Result_test_1, Result_test_2]
result_one_file_2 = pd.concat(frame, axis=1)

frame = [result_one_file_1, result_one_file_2]
result_one_file_total = pd.concat(frame, axis=1)

In [188]:
result_dataframe = pd.DataFrame(result_one_file_total).T
result_dataframe.reset_index(drop=True, inplace=True)
result_dataframe.index = result_dataframe.index + 1
result_dataframe.to_excel("temp.xlsx", index=False)